In [375]:
import pyomo.environ as pyo
import pandas as pd
from pyomo.environ import ConcreteModel, Var, Objective, Constraint, SolverFactory
import pandas as pd

In [376]:
demand = pd.read_csv('load.csv',header=None)
demand.rename(columns={0:"timestep", 1:"load_MW"}, inplace=True)
demand

timestep  load_MW
0       t1    83115
1       t2    71169
2       t3    66729
3       t4    61442
4       t5    60430
5       t6    57013
6       t7    52048
7       t8    48701
8       t9    43981
9      t10    40498

In [377]:
cf = pd.read_csv('capacity_factors.csv')
cf.rename(columns={cf.columns[0]:"tech"}, inplace=True)
cf[["t1", "t2", "t3", "t4", "t5","t6", "t7", "t8", "t9", "t10"]] = cf[["t1", "t2", "t3", "t4", "t5","t6", "t7", "t8", "t9", "t10"]].apply(pd.to_numeric)
cf

tech    t1    t2    t3    t4    t5    t6    t7    t8    t9   t10
0       CCGT  0.59  0.63  0.59  0.61  0.67  0.61  0.63  0.67  0.62  0.64
1  GT_GasOil  0.63  0.69  0.68  0.68  0.70  0.63  0.68  0.61  0.70  0.70
2      Hydro  0.78  0.79  0.69  0.74  0.71  0.83  0.80  0.70  0.78  0.82
3       Coal  0.82  0.92  0.88  0.82  0.93  0.83  0.82  0.82  0.80  0.85
4    Lignite  0.81  0.86  0.77  0.77  0.82  0.81  0.84  0.77  0.86  0.86
5    Nuclear  0.95  0.96  0.87  0.89  0.92  0.94  0.97  0.87  0.88  0.92
6       Wind  0.17  0.16  0.20  0.21  0.31  0.31  0.21  0.24  0.34  0.17
7      Solar  0.24  0.05  0.09  0.17  0.16  0.15  0.12  0.03  0.00  0.00

In [378]:
duration = pd.read_csv('duration.csv',header=None)
duration.rename(columns={0:"timestep", 1:"length"}, inplace=True)
duration

timestep  length
0       t1     102
1       t2     962
2       t3     962
3       t4     962
4       t5     962
5       t6     962
6       t7     962
7       t8     962
8       t9     962
9      t10     962

In [379]:
tech_data = pd.read_csv('tech_data.csv',header=None)
tech_data.rename(columns={0:"tech", 1:"cap_MW",2:"eta",3:"fuel_p",4:"c_var_other",5:"emf"}, inplace=True)
tech_data.drop([0,1],inplace=True)
tech_data[["cap_MW", "eta", "fuel_p", "c_var_other", "emf"]] = tech_data[["cap_MW", "eta", "fuel_p", "c_var_other", "emf"]].apply(pd.to_numeric)
tech_data.reset_index(drop=True, inplace=True)
tech_data

tech  cap_MW   eta  fuel_p  c_var_other     emf
0       CCGT   30000  0.54    19.0          1.5  0.2048
1  GT_GasOil    4400  0.28    19.0          1.5  0.2048
2      Hydro    5200  1.00     0.0          1.5  0.0000
3       Coal   22500  0.42     7.4          2.6  0.3420
4    Lignite   21000  0.37     3.4          3.0  0.3996
5    Nuclear    8400  0.33     1.8          0.7  0.0000
6       Wind   61000  1.00     0.0          1.4  0.0000
7      Solar   46500  1.00     0.0          1.0  0.0000

In [246]:
# tech_data['marginal_price'] = round((tech_data['fuel_p']/tech_data['eta'] + tech_data['c_var_other'] + 0*tech_data['emf']/tech_data['eta']),2)
# tech_data['Actual_Capacity'] = tech_data['cap_MW']*cf['t1']
# tech_data

In [380]:
# Define the cost functions for each generator
def marginal_price(generator,co2_price):
    # Cost function for generator 1
    # You can modify this function according to your specific cost model
    df = tech_data[tech_data['tech'] == generator]
    return round((df['fuel_p']/df['eta'] + df['c_var_other'] + co2_price*df['emf']/df['eta']).values[0],2)


Create a generic model `m` for all timesteps

In [381]:
# Create a concrete Pyomo model
m = pyo.ConcreteModel()
m.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)

We create a set $S$ for the technologies:

In [382]:
m.S = pyo.Set(initialize=tech_data['tech'])

In [383]:
m.S.pprint()

S : Size=1, Index=None, Ordered=Insertion
    Key  : Dimen : Domain : Size : Members
    None :     1 :    Any :    8 : {'CCGT', 'GT_GasOil', 'Hydro', 'Coal', 'Lignite', 'Nuclear', 'Wind', 'Solar'}


Define the decision variable generators

In [384]:
# Define the decision variable for each generator
m.generators = Var(m.S, domain=pyo.NonNegativeReals)

In [385]:
# Define the objective function
m.cost = Objective(expr=sum(marginal_price(s,0)*m.generators[s] for s in m.S),
                      sense=pyo.minimize)

In [386]:
m.cost.pprint()

cost : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 36.69*generators[CCGT] + 69.36*generators[GT_GasOil] + 1.5*generators[Hydro] + 20.22*generators[Coal] + 12.19*generators[Lignite] + 6.15*generators[Nuclear] + 1.4*generators[Wind] + generators[Solar]


Create dictionary of models for all timesteps

In [387]:
#create dictionary of models

models = {}
for i in range(len(duration)):
    models[duration['timestep'][i]] = m.clone()

timesteps = list(models.keys())

In [388]:
for i in timesteps:
    model = models[i]
    @model.Constraint(model.S)
    def generator_limit(model, s):
        return model.generators[s] <= cf[cf['tech'] == s][i].values[0]*tech_data[tech_data['tech'] == s].cap_MW.values[0]
    models[i] = model
    models[i].demand_constraint = Constraint(expr=sum(models[i].generators[s] for s in models[i].S) == demand[demand.timestep == i].load_MW.values[0])

In [390]:
models['t2'].generator_limit.pprint()

generator_limit : Size=8, Index=S, Active=True
    Key       : Lower : Body                  : Upper              : Active
         CCGT :  -Inf :      generators[CCGT] :            18900.0 :   True
         Coal :  -Inf :      generators[Coal] :            20700.0 :   True
    GT_GasOil :  -Inf : generators[GT_GasOil] : 3035.9999999999995 :   True
        Hydro :  -Inf :     generators[Hydro] :             4108.0 :   True
      Lignite :  -Inf :   generators[Lignite] :            18060.0 :   True
      Nuclear :  -Inf :   generators[Nuclear] :             8064.0 :   True
        Solar :  -Inf :     generators[Solar] :             2325.0 :   True
         Wind :  -Inf :      generators[Wind] :             9760.0 :   True


In [391]:
models['t2'].demand_constraint.pprint()

demand_constraint : Size=1, Index=None, Active=True
    Key  : Lower   : Body                                                                                                                                                               : Upper   : Active
    None : 71169.0 : generators[CCGT] + generators[GT_GasOil] + generators[Hydro] + generators[Coal] + generators[Lignite] + generators[Nuclear] + generators[Wind] + generators[Solar] : 71169.0 :   True


In [208]:
m.generator_limit.pprint()

generator_limit : Size=8, Index=S, Active=True
    Key       : Lower : Body                  : Upper   : Active
         CCGT :  -Inf :      generators[CCGT] : 17700.0 :   True
         Coal :  -Inf :      generators[Coal] : 18450.0 :   True
    GT_GasOil :  -Inf : generators[GT_GasOil] :  2772.0 :   True
        Hydro :  -Inf :     generators[Hydro] :  4056.0 :   True
      Lignite :  -Inf :   generators[Lignite] : 17010.0 :   True
      Nuclear :  -Inf :   generators[Nuclear] :  7980.0 :   True
        Solar :  -Inf :     generators[Solar] : 11160.0 :   True
         Wind :  -Inf :      generators[Wind] : 10370.0 :   True


In [393]:
for i in timesteps:
    SolverFactory('cbc').solve(models[i]).write()

# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 1178175.31
  Upper bound: 1178175.31
  Number of objectives: 1
  Number of constraints: 10
  Number of variables: 9
  Number of nonzeros: 8
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.41
  Wallclock time: 0.41
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number o

In [397]:
print("Optimal Generator Outputs:")
for s in m.S:
    print(f"{s}: {models['t10'].generators[s].value}")

print("Total Cost:", models['t10'].cost.expr())

Optimal Generator Outputs:
CCGT: 0.0
GT_GasOil: 0.0
Hydro: 4264.0
Coal: 76.0
Lignite: 18060.0
Nuclear: 7728.0
Wind: 10370.0
Solar: 0.0
Total Cost: 290129.32


In [261]:
m.pprint()

1 Set Declarations
    S : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {'CCGT', 'GT_GasOil', 'Hydro', 'Coal', 'Lignite', 'Nuclear', 'Wind', 'Solar'}

1 Var Declarations
    generators : Size=8, Index=S
        Key       : Lower : Value   : Upper : Fixed : Stale : Domain
             CCGT :     0 : 14089.0 :  None : False : False : NonNegativeReals
             Coal :     0 : 18450.0 :  None : False : False : NonNegativeReals
        GT_GasOil :     0 :     0.0 :  None : False : False : NonNegativeReals
            Hydro :     0 :  4056.0 :  None : False : False : NonNegativeReals
          Lignite :     0 : 17010.0 :  None : False : False : NonNegativeReals
          Nuclear :     0 :  7980.0 :  None : False : False : NonNegativeReals
            Solar :     0 : 11160.0 :  None : False : False : NonNegativeReals
             Wind :     0 : 10370.0 :  None : False : False : NonNegativeReals

1 Objective Decl

In [398]:
pd.Series(models['t1'].generators.get_values()) #turn dictionary into pd Series

CCGT         14089.0
GT_GasOil        0.0
Hydro         4056.0
Coal         18450.0
Lignite      17010.0
Nuclear       7980.0
Wind         10370.0
Solar        11160.0
dtype: float64

In [422]:
dispatch = pd.Series(models[timesteps[0]].generators.get_values(),name=timesteps[0]).to_frame()
for i in timesteps[1:]:
    d = pd.Series(models[i].generators.get_values(),name=i).to_frame()
    dispatch = dispatch.join(d)
dispatch

t1       t2       t3       t4       t5       t6       t7  \
CCGT       14089.0   8152.0   3478.0      0.0      0.0      0.0      0.0   
GT_GasOil      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
Hydro       4056.0   4108.0   3588.0   3848.0   3692.0   4316.0   4160.0   
Coal       18450.0  20700.0  19800.0  13233.0   5440.0   1906.0   3710.0   
Lignite    17010.0  18060.0  16170.0  16170.0  17220.0  17010.0  17640.0   
Nuclear     7980.0   8064.0   7308.0   7476.0   7728.0   7896.0   8148.0   
Wind       10370.0   9760.0  12200.0  12810.0  18910.0  18910.0  12810.0   
Solar      11160.0   2325.0   4185.0   7905.0   7440.0   6975.0   5580.0   

                t8       t9      t10  
CCGT           0.0      0.0      0.0  
GT_GasOil      0.0      0.0      0.0  
Hydro       3640.0   4056.0   4264.0  
Coal        5548.0      0.0     76.0  
Lignite    16170.0  11793.0  18060.0  
Nuclear     7308.0   7392.0   7728.0  
Wind       14640.0  20740.0  10370.0  
Solar       1395.0      0.0      0.0

In [423]:
a = pd.Series(models['t5'].generators.get_values()).to_frame()
a

0
CCGT           0.0
GT_GasOil      0.0
Hydro       3692.0
Coal        5440.0
Lignite    17220.0
Nuclear     7728.0
Wind       18910.0
Solar       7440.0

In [530]:
dispatch = pd.Series(models[timesteps[0]].generators.get_values(),name=timesteps[0]).to_frame()
for i in timesteps[1:]:
    d = pd.Series(models[i].generators.get_values(),name=i).to_frame()
    dispatch = dispatch.join(d)
dispatch

t1       t2       t3       t4       t5       t6       t7  \
CCGT       14089.0   8152.0   3478.0      0.0      0.0      0.0      0.0   
GT_GasOil      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
Hydro       4056.0   4108.0   3588.0   3848.0   3692.0   4316.0   4160.0   
Coal       18450.0  20700.0  19800.0  13233.0   5440.0   1906.0   3710.0   
Lignite    17010.0  18060.0  16170.0  16170.0  17220.0  17010.0  17640.0   
Nuclear     7980.0   8064.0   7308.0   7476.0   7728.0   7896.0   8148.0   
Wind       10370.0   9760.0  12200.0  12810.0  18910.0  18910.0  12810.0   
Solar      11160.0   2325.0   4185.0   7905.0   7440.0   6975.0   5580.0   

                t8       t9      t10  
CCGT           0.0      0.0      0.0  
GT_GasOil      0.0      0.0      0.0  
Hydro       3640.0   4056.0   4264.0  
Coal        5548.0      0.0     76.0  
Lignite    16170.0  11793.0  18060.0  
Nuclear     7308.0   7392.0   7728.0  
Wind       14640.0  20740.0  10370.0  
Solar       1395.0      0.0      0.0

In [512]:
dispatch_costs = duration
dispatch_costs['load_MW'] = demand['load_MW']
dispatch_costs['electricity_price'] = [models[t].dual[models[t].demand_constraint] for t in timesteps]
dispatch_costs['expenditure'] = dispatch_costs['load_MW']*dispatch_costs['electricity_price']*dispatch_costs['length']
dispatch_costs

timestep  length  electricity_price  load_MW   expenditure
0       t1     102              36.69    83115  3.110479e+08
1       t2     962              36.69    71169  2.511965e+09
2       t3     962              36.69    66729  2.355252e+09
3       t4     962              20.22    61442  1.195148e+09
4       t5     962              20.22    60430  1.175463e+09
5       t6     962              20.22    57013  1.108996e+09
6       t7     962              20.22    52048  1.012419e+09
7       t8     962              20.22    48701  9.473143e+08
8       t9     962              12.19    43981  5.157555e+08
9      t10     962              20.22    40498  7.877525e+08

In [460]:
print("Total annual expenditure [billion €]:", round((dispatch_costs.expenditure).sum()/1e9,3))
print("Average electricity price [€/MWh]:", round(((dispatch_costs.electricity_price*dispatch_costs.length).sum()/dispatch_costs.length.sum()),2))

Total annual expenditure [billion €]: 11.921
Average electricity price [€/MWh]: 23.15


In [465]:
dispatch

t1       t2       t3       t4       t5       t6       t7  \
CCGT       14089.0   8152.0   3478.0      0.0      0.0      0.0      0.0   
GT_GasOil      0.0      0.0      0.0      0.0      0.0      0.0      0.0   
Hydro       4056.0   4108.0   3588.0   3848.0   3692.0   4316.0   4160.0   
Coal       18450.0  20700.0  19800.0  13233.0   5440.0   1906.0   3710.0   
Lignite    17010.0  18060.0  16170.0  16170.0  17220.0  17010.0  17640.0   
Nuclear     7980.0   8064.0   7308.0   7476.0   7728.0   7896.0   8148.0   
Wind       10370.0   9760.0  12200.0  12810.0  18910.0  18910.0  12810.0   
Solar      11160.0   2325.0   4185.0   7905.0   7440.0   6975.0   5580.0   

                t8       t9      t10  
CCGT           0.0      0.0      0.0  
GT_GasOil      0.0      0.0      0.0  
Hydro       3640.0   4056.0   4264.0  
Coal        5548.0      0.0     76.0  
Lignite    16170.0  11793.0  18060.0  
Nuclear     7308.0   7392.0   7728.0  
Wind       14640.0  20740.0  10370.0  
Solar       1395.0      0.0      0.0

In [513]:
dispatch_emissions = dispatch

In [564]:
for i in timesteps:
    dispatch_emissions[i] = (tech_data.set_index(dispatch.index)['emf'].values*dispatch[i]/tech_data.set_index(dispatch.index)['eta'])*duration[duration['timestep'] == 't1'].length.values[0]

dispatch_emissions  #total emissions per timestep

t1            t2            t3            t4  \
CCGT       5.450251e+05  3.153556e+05  1.345445e+05  0.000000e+00   
GT_GasOil  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
Hydro      0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
Coal       1.532404e+06  1.719283e+06  1.644531e+06  1.099095e+06   
Lignite    1.873822e+06  1.989490e+06  1.781287e+06  1.781287e+06   
Nuclear    0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
Wind       0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
Solar      0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   

                     t5            t6         t7            t8          t9  \
CCGT       0.000000e+00  0.000000e+00        0.0  0.000000e+00        0.00   
GT_GasOil  0.000000e+00  0.000000e+00        0.0  0.000000e+00        0.00   
Hydro      0.000000e+00  0.000000e+00        0.0  0.000000e+00        0.00   
Coal       4.518309e+05  1.583069e+05   308142.0  4.608010e+05        0.00   
Lignite    1.896955e+06  1.873822e+06  1943222.4  1.781287e+06  1299116.88   
Nuclear    0.000000e+00  0.000000e+00        0.0  0.000000e+00        0.00   
Wind       0.000000e+00  0.000000e+00        0.0  0.000000e+00        0.00   
Solar      0.000000e+00  0.000000e+00        0.0  0.000000e+00        0.00   

                    t10  
CCGT       0.000000e+00  
GT_GasOil  0.000000e+00  
Hydro      0.000000e+00  
Coal       6.312343e+03  
Lignite    1.989490e+06  
Nuclear    0.000000e+00  
Wind       0.000000e+00  
Solar      0.000000e+00

In [563]:
print('Total emissions [million t/CO2]:', round(dispatch_emissions.sum().sum()/1e6,2))

Total emissions [million t/CO2]: 26.59


In [220]:
pd.Series(m.dual.values(), m.dual.keys())

generator_limit[CCGT]          0.00
generator_limit[Coal]        -16.47
generator_limit[GT_GasOil]     0.00
generator_limit[Hydro]       -35.19
generator_limit[Lignite]     -24.50
generator_limit[Nuclear]     -30.54
generator_limit[Solar]       -35.69
generator_limit[Wind]        -35.29
[None]                        36.69
dtype: float64

In [221]:
pd.Series({s: m.dual[m.generator_limit[s]] for s in m.S}) #create a dictionary with list comprehension

CCGT          0.00
GT_GasOil     0.00
Hydro       -35.19
Coal        -16.47
Lignite     -24.50
Nuclear     -30.54
Wind        -35.29
Solar       -35.69
dtype: float64

In [567]:
pd.Series({s: models['t1'].dual[models['t1'].generator_limit[s]] for s in models['t1'].S}).to_frame()

0
CCGT        0.00
GT_GasOil   0.00
Hydro     -35.19
Coal      -16.47
Lignite   -24.50
Nuclear   -30.54
Wind      -35.29
Solar     -35.69

In [569]:
dual_prices = pd.Series({s: models[timesteps[0]].dual[models[timesteps[0]].generator_limit[s]] for s in models[timesteps[0]].S},name=timesteps[0]).to_frame()
for i in timesteps[1:]:
    dp = pd.Series({s: models[i].dual[models[i].generator_limit[s]] for s in models[i].S},name=i).to_frame()
    dual_prices = dual_prices.join(dp)
dual_prices

t1     t2     t3     t4     t5     t6     t7     t8     t9  \
CCGT        0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   
GT_GasOil   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   0.00   
Hydro     -35.19 -35.19 -35.19 -18.72 -18.72 -18.72 -18.72 -18.72 -10.69   
Coal      -16.47 -16.47 -16.47   0.00   0.00   0.00   0.00   0.00   0.00   
Lignite   -24.50 -24.50 -24.50  -8.03  -8.03  -8.03  -8.03  -8.03   0.00   
Nuclear   -30.54 -30.54 -30.54 -14.07 -14.07 -14.07 -14.07 -14.07  -6.04   
Wind      -35.29 -35.29 -35.29 -18.82 -18.82 -18.82 -18.82 -18.82 -10.79   
Solar     -35.69 -35.69 -35.69 -19.22 -19.22 -19.22 -19.22 -19.22 -11.19   

             t10  
CCGT        0.00  
GT_GasOil   0.00  
Hydro     -18.72  
Coal        0.00  
Lignite    -8.03  
Nuclear   -14.07  
Wind      -18.82  
Solar     -19.22

### MISC

In [ ]:
models['t1'].cost.pprint()

cost : Size=1, Index=None, Active=True
    Key  : Active : Sense    : Expression
    None :   True : minimize : 36.69*generators[CCGT] + 69.36*generators[GT_GasOil] + 1.5*generators[Hydro] + 20.22*generators[Coal] + 12.19*generators[Lignite] + 6.15*generators[Nuclear] + 1.4*generators[Wind] + generators[Solar]


In [ ]:
models['t1'].cost.expr()

1178175.3099999998

In [250]:
# Define the set of generators
#generators = ['CCGT', 'GT_GasOil', 'Hydro', 'Coal', 'Lignite', 'Nuclear', 'Wind', 'Solar']  # Add more generator names as needed

In [ ]:
# Define the fixed demand
load = demand[demand.timestep == 't1'].load_MW[0]

In [ ]:
@m.Constraint(m.S)
def generator_limit(m, s):
    return m.generators[s] <= cf[cf['tech'] == s]['t1'].values[0]*tech_data[tech_data['tech'] == s].cap_MW.values[0]

In [ ]:
m.generator_limit.pprint()

generator_limit : Size=8, Index=S, Active=True
    Key       : Lower : Body                  : Upper   : Active
         CCGT :  -Inf :      generators[CCGT] : 17700.0 :   True
         Coal :  -Inf :      generators[Coal] : 18450.0 :   True
    GT_GasOil :  -Inf : generators[GT_GasOil] :  2772.0 :   True
        Hydro :  -Inf :     generators[Hydro] :  4056.0 :   True
      Lignite :  -Inf :   generators[Lignite] : 17010.0 :   True
      Nuclear :  -Inf :   generators[Nuclear] :  7980.0 :   True
        Solar :  -Inf :     generators[Solar] : 11160.0 :   True
         Wind :  -Inf :      generators[Wind] : 10370.0 :   True


In [ ]:
# Define the constraint for fixed demand
m.demand_constraint = Constraint(expr=sum(m.generators[s] for s in m.S) == demand.load_MW[0])

# Solve the optimization problem
solver = SolverFactory('glpk')  # Choose the solver you have installed (e.g., 'glpk', 'gurobi', 'cplex')
result = solver.solve(m)

# Print the optimal generator outputs and total cost
print("Optimal Generator Outputs:")
for s in m.S:
    print(f"{s}: {m.generators[s].value}")

print("Total Cost:", m.cost.expr())

Optimal Generator Outputs:
CCGT: 14089.0
GT_GasOil: 0.0
Hydro: 4056.0
Coal: 18450.0
Lignite: 17010.0
Nuclear: 7980.0
Wind: 10370.0
Solar: 11160.0
Total Cost: 1178175.3099999998


In [ ]:
m.demand_constraint.pprint()

demand_constraint : Size=1, Index=None, Active=True
    Key  : Lower   : Body                                                                                                                                                               : Upper   : Active
    None : 83115.0 : generators[CCGT] + generators[GT_GasOil] + generators[Hydro] + generators[Coal] + generators[Lignite] + generators[Nuclear] + generators[Wind] + generators[Solar] : 83115.0 :   True


In [ ]:
m.pprint()

1 Set Declarations
    S : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    8 : {'CCGT', 'GT_GasOil', 'Hydro', 'Coal', 'Lignite', 'Nuclear', 'Wind', 'Solar'}

1 Var Declarations
    generators : Size=8, Index=S
        Key       : Lower : Value   : Upper : Fixed : Stale : Domain
             CCGT :     0 : 14089.0 :  None : False : False : NonNegativeReals
             Coal :     0 : 18450.0 :  None : False : False : NonNegativeReals
        GT_GasOil :     0 :     0.0 :  None : False : False : NonNegativeReals
            Hydro :     0 :  4056.0 :  None : False : False : NonNegativeReals
          Lignite :     0 : 17010.0 :  None : False : False : NonNegativeReals
          Nuclear :     0 :  7980.0 :  None : False : False : NonNegativeReals
            Solar :     0 : 11160.0 :  None : False : False : NonNegativeReals
             Wind :     0 : 10370.0 :  None : False : False : NonNegativeReals

1 Objective Decl